[View in Colaboratory](https://colab.research.google.com/github/ameasure/try_git/blob/master/Finetune.ipynb)

Transfer Learning with Finetune 

https://https://github.com/IndicoDataSolutions/finetune

In [13]:
!pip install finetune git+git://github.com/IndicoDataSolutions/finetune.git@92b28ecc1db0e33838995ea792b5d3b6f3dd7686
!python -m spacy download en_core_web_sm
!python -m spacy link en_core_web_sm en
!pip install tables
!pip install xlrd
!wget 'https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx'

  Cloning git://github.com/IndicoDataSolutions/finetune.git (to revision 92b28ecc1db0e33838995ea792b5d3b6f3dd7686) to /tmp/pip-req-build-iezucrws
    100% |████████████████████████████████| 51kB 3.0MB/s 
    100% |████████████████████████████████| 8.9MB 830kB/s 
    100% |████████████████████████████████| 31.2MB 1.4MB/s 
    100% |████████████████████████████████| 61kB 15.6MB/s 
    100% |████████████████████████████████| 143kB 28.2MB/s 
    100% |████████████████████████████████| 153kB 27.8MB/s 
  Running setup.py bdist_wheel for finetune ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8e/8d/9d/3fc6353867db3b2f034897211c30d2dcb13ed46e724447d20d
  Running setup.py bdist_wheel for IndicoIo ... - \ done
  Stored in directory: /root/.cache/pip/wheels/92/45/b8/3cc5048729333ee7fafff88522c9537d999b176453ca127798
  Running setup.py bdist_wheel for ftfy ... - \ done
  Stored in directory: /root/.cache/pip/wheels/e6/42/b1/0d7f9445a9df8920548ab9bb953d6587067c34bc47e5be962

In [14]:
import pandas as pd

df = pd.read_excel('msha.xlsx')
df['ACCIDENT_YEAR'] = df['ACCIDENT_DT'].apply(lambda x: x.year)
df['ACCIDENT_YEAR'].value_counts()
df_train = df[df['ACCIDENT_YEAR'].isin([2010, 2011])].copy()
df_valid = df[df['ACCIDENT_YEAR'] == 2012].copy()
print('training rows:', len(df_train))
print('validation rows:', len(df_valid))

training rows: 18681
validation rows: 9032


In [15]:
from finetune import Classifier

# max_length is the maximum number of words we will use from each narrative
# behind the scenes finetune is setting aside 5% of the training data for validation
model = Classifier(verbose=False, batch_size=32, max_length=90, n_epochs=4)
model.fit(df_train['NARRATIVE'], df_train['INJ_BODY_PART'])

/usr/local/lib/python3.6/dist-packages/finetune/encoding.py:294: UserWarning: Some examples are longer than the max_length. Please trim documents or increase `max_length`. Fallback behaviour is to use the first 88 byte-pair encoded tokens
  "Fallback behaviour is to use the first {} byte-pair encoded tokens".format(max_length - 2)
/usr/local/lib/python3.6/dist-packages/finetune/encoding.py:233: UserWarning: Document is longer than max length allowed, trimming document to 90 tokens.
  max_length


Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use `tf.variables_initializer` instead.
Train loss: 2.02428658947826	 Validation loss: 1.4960562558931962
Train loss: 1.1733399682670937	 Validation loss: 1.326524703531812
Train loss: 0.8657235422376427	 Validation loss: 1.1786635609289138
Train loss: 0.8157964310963368	 Validation loss: 1.0611117305191342
Train loss: 0.7282302780120924	 Validation loss: 0.9721394627659866
Train loss: 0.6319493325652035	 Validation loss: 0.9009963018470547
Train loss: 0.6187396777123942	 Validation loss: 0.8520151351536146
Train loss: 0.5823027279206591	 Validation loss: 0.8105794074323073
Train loss: 0.5131763383288986	 Validation loss: 0.7771654124238514
Train loss: 0.4661312149687483	 Validation loss: 0.758779292345116
Train loss: 0.47322449278832546	 Validation loss: 0.7405749610013915
Train loss: 0.42416194645229194	 Validation loss: 0.7260644953524004
Train loss: 0.36603444547124886	 Validation loss: 0.7146084719484589
Train loss: 0.3132562995922706	 Validation loss: 0

In [16]:
# generate preditions
df_valid['PREDICTED_PART'] = model.predict(df_valid['NARRATIVE'].values)
# look at a sample
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART']].sample(5)

,NARRATIVE,INJ_BODY_PART,PREDICTED_PART
24755,The EE was dragging water hose up the slope wh...,KNEE/PATELLA,KNEE/PATELLA
5720,The man was pushing up on a panel on a Kennedy...,WRIST,WRIST
14880,Loud noise / Hearing loss.,EAR(S) INTERNAL & HEARING,EAR(S) INTERNAL & HEARING
23610,Employee was sliding an electric motor from a ...,FINGER(S)/THUMB,FINGER(S)/THUMB
18036,Standard Threshold Shift in employee's hearing...,EAR(S) INTERNAL & HEARING,EAR(S) INTERNAL & HEARING


In [17]:
pd.options.display.max_colwidth=500

df_valid['PROB_DICT'] = model.predict_proba(df_valid['NARRATIVE'].values)
df_valid[['NARRATIVE', 'PREDICTED_PART', 'PROB_DICT']].head()

,NARRATIVE,PREDICTED_PART,PROB_DICT
2,"Employee, parked s/c on grade at 16-Block #3 Entry Spad #3868. S/c slid approx. 3' pinning oper. between s/c & rib, employee had set park brake and got off machine to move roof bolter cable.",SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),"{'ABDOMEN/INTERNAL ORGANS': 0.021482423, 'ANKLE': 0.005168089, 'ARM, MULTIPLE PARTS': 0.0011740514, 'ARM,NEC': 0.0096712345, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.055421397, 'BODY PARTS, NEC': 0.0012622344, 'BODY SYSTEMS': 0.00076493644, 'BRAIN': 0.004081469, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.104457006, 'EAR(S) EXTERNAL': 0.0014992885, 'EAR(S) INTERNAL & EXTERNAL': 0.000327146, 'EAR(S) INTERNAL & HEARING': 0.00030815712, 'ELBOW': 0.008493464, 'EYE(S) OPTIC NERVE/VISON': 0.0009638..."
5,Possible heart attack.,BODY SYSTEMS,"{'ABDOMEN/INTERNAL ORGANS': 0.0006787256, 'ANKLE': 7.976627e-05, 'ARM, MULTIPLE PARTS': 9.517138e-05, 'ARM,NEC': 0.00037252132, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 6.8991416e-05, 'BODY PARTS, NEC': 0.0001418036, 'BODY SYSTEMS': 0.95541155, 'BRAIN': 0.0005341452, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.010710106, 'EAR(S) EXTERNAL': 0.00011770584, 'EAR(S) INTERNAL & EXTERNAL': 4.2028423e-05, 'EAR(S) INTERNAL & HEARING': 0.00037901106, 'ELBOW': 3.1924483e-05, 'EYE(S) OPTIC NERVE/VISON': 7..."
8,Employee was cleaning up plant spillage into a fork lift mounted self-dumping hopper. Latch on hopper was not operating properly and employee repeatedly lifted on hopper to unlatch it. Employee reported that his right shoulder hurt from the lifting at end of shift. Placed on light duty with no heavy lifting.,SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),"{'ABDOMEN/INTERNAL ORGANS': 6.264959e-06, 'ANKLE': 1.0312645e-05, 'ARM, MULTIPLE PARTS': 7.0406195e-06, 'ARM,NEC': 5.059311e-05, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.00039924055, 'BODY PARTS, NEC': 4.5551865e-06, 'BODY SYSTEMS': 1.060816e-05, 'BRAIN': 1.7900391e-05, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 1.8342738e-05, 'EAR(S) EXTERNAL': 3.0551022e-05, 'EAR(S) INTERNAL & EXTERNAL': 3.738909e-06, 'EAR(S) INTERNAL & HEARING': 9.614982e-06, 'ELBOW': 2.2118455e-05, 'EYE(S) OPTIC NERVE/VISO..."
12,"Employee was putting a drag on a shuttle car, when he lifted one end and another employee lifted the other end, the drag smashed the end of his middle finger on his left hand. He was taken to the hospital and received 12 stitches in his finger and returned to work.",FINGER(S)/THUMB,"{'ABDOMEN/INTERNAL ORGANS': 2.3168336e-06, 'ANKLE': 2.6768614e-06, 'ARM, MULTIPLE PARTS': 8.590277e-06, 'ARM,NEC': 6.8721884e-06, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 2.8429595e-06, 'BODY PARTS, NEC': 3.817813e-06, 'BODY SYSTEMS': 7.2016483e-06, 'BRAIN': 2.312304e-06, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 1.59452e-06, 'EAR(S) EXTERNAL': 7.006113e-06, 'EAR(S) INTERNAL & EXTERNAL': 1.6748319e-06, 'EAR(S) INTERNAL & HEARING': 5.0143735e-06, 'ELBOW': 1.3061186e-05, 'EYE(S) OPTIC NERVE/VISON..."
27,While using a cutting torch to remove a bearing the employee was burned by the torch tip after another employee tripped over the torch hose causing the torch flame to contact his left hand.,HAND (NOT WRIST OR FINGERS),"{'ABDOMEN/INTERNAL ORGANS': 4.601311e-06, 'ANKLE': 1.6828939e-05, 'ARM, MULTIPLE PARTS': 2.4594832e-05, 'ARM,NEC': 2.8724098e-05, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 1.9359019e-05, 'BODY PARTS, NEC': 4.4981607e-06, 'BODY SYSTEMS': 0.000111284644, 'BRAIN': 5.086649e-06, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 5.84468e-05, 'EAR(S) EXTERNAL': 6.5714225e-06, 'EAR(S) INTERNAL & EXTERNAL': 7.561547e-06, 'EAR(S) INTERNAL & HEARING': 2.4453151e-05, 'ELBOW': 1.0785815e-05, 'EYE(S) OPTIC NERVE/VIS..."


In [18]:
# function that takes a row of our dataframe and returns the predicted probability
def get_probability(row):
    predicted_part = row['PREDICTED_PART']
    probability_dict = row['PROB_DICT']
    return probability_dict[predicted_part]

# apply get_probability to each row in our dataframe and store the result
df_valid['PREDICTED_PROB'] = df_valid.apply(func=get_probability, axis=1)
# take a peak at what we get
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART', 'PREDICTED_PROB']].sample(5).head()

,NARRATIVE,INJ_BODY_PART,PREDICTED_PART,PREDICTED_PROB
32972,Facts are at time of this report (taken from statement by Anderson Count Sheriffs Dept). 2 males on haulage road from mine got into a fight? and one shot the other with a 40 cal Taurus handgun. Victim is alive at this time. Neither man is employed by company.,ABDOMEN/INTERNAL ORGANS,MULTIPLE PARTS (MORE THAN ONE MAJOR),0.639339
14586,"While attempting to secure a nut on a bolt on the bottom of the BB float cell, the employee fell from ladder when he lost his balance. The employee was standing on the second step of a 4 foot step ladder on the ground floor on the East side of the Float Plant. He fell to the ground, striking his left side (rib area), on the water hose of the nearby water cannon.",CHEST (RIBS/BREAST BONE/CHEST ORGNS),CHEST (RIBS/BREAST BONE/CHEST ORGNS),0.951890
36909,EE was changing batteries on a scoop and the battery lid dropped down and caught his left thumb. Sutures were required no lost time.,FINGER(S)/THUMB,FINGER(S)/THUMB,0.996906
21789,Miner was bolting and the leg extended rapidly. The miner tried to catch the machine and got his hand in between the wall and the handle of the machine.,HAND (NOT WRIST OR FINGERS),HAND (NOT WRIST OR FINGERS),0.519798
15752,"A driver lost control of his haul truck resulting in a tip-over. According to the driver, he was wearing his seat belt, yet he stated that he impacted the left side of the cab. The driver stated that he had pain on his left side, left lower back and left shoulder. No external bruising, or abrasion was visible. X-ray and CT scan showed no internal injuries.","TRUNK, MULTIPLE PARTS",BACK (MUSCLES/SPINE/S-CORD/TAILBONE),0.517057


In [19]:
from sklearn.metrics import accuracy_score, f1_score

mf1 = f1_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'], average='macro')
acc = accuracy_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'])
print('macro-f1:', mf1)
print('accuracy:', acc)

macro-f1: 0.6602087283637166
accuracy: 0.8237378210806023


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
